# `numpy` - numerical python 

https://numpy.org/

Co o sobě říká `numpy`:

> **POWERFUL N-DIMENSIONAL ARRAYS** Fast and versatile, the NumPy vectorization, indexing, and broadcasting concepts are the de-facto standards of array computing today.

> **NUMERICAL COMPUTING TOOLS** NumPy offers comprehensive mathematical functions, random number generators, linear algebra routines, Fourier transforms, and more.

> **OPEN SOURCE** Distributed under a liberal BSD license, NumPy is developed and maintained publicly on GitHub by a vibrant, responsive, and diverse community.

> **INTEROPERABLE** NumPy supports a wide range of hardware and computing platforms, and plays well with distributed, GPU, and sparse array libraries.

> **PERFORMANT** The core of NumPy is well-optimized C code. Enjoy the flexibility of Python with the speed of compiled code.

> **EASY TO USE** NumPy’s high level syntax makes it accessible and productive for programmers from any background or experience level.

## `numpy` arrays

Základním stavebním kamenem jsou vícerozměrná pole. Oproti např. typu `list` mají pevnou velikost a mohou obsahovat prvky pouze jednoho typu. Vytváříme je pomocí zabudovaných funkci, jako je třeba celkem univerzální `numpy.array` nebo specializovaných, např. `numpy.zeros`, `numpy.ones`, `numpy.linspace`.

Pole mají řadu atributů. Mezi ty základní patří `.shape`, `.size`, `.dtype`. Jejich význam by mohl být zřejmá z následujících příkladů.

In [ ]:
import numpy as np

a = np.array([1, 2, 3], dtype=np.double)
print(a)

a = np.zeros(shape=(3, 2), dtype=np.complex128)
print(a)

a = np.linspace(0, 1, 11)
print(a)

a = np.array(range(18), dtype=int).reshape(3, 6)
print(a)

a = np.array(range(8), dtype=int).reshape(2, 2, 2)
print(a)

Mezi `numpy` poli jsou definovány všechny rozumné operace, včetně řady algebraických (násobení matic, skalární a vektorový součin, atd.). Řada z nich má asociovaný i nějaký operátor.

In [ ]:
a = np.array([1, 2, 3])
b = np.array([1, 1, 1])

np.matmul(a, b) # a @ b # skalární součin / maticové násobení

In [ ]:
c = 2 * a + b**2
print(c)

In [ ]:
a * b # element-wise multiplication

## Broadcasting

Broadcasting je způsob, jakým `numpy` řeší operace s poli různých velikostí. Nejsnáze se to ilustruje na násobení pole skalárem (číslem)

In [ ]:
import numpy as np

a = np.array(range(10))
b = 2

c = a * b
print(c.shape, c)

`numpy` 'natáhne' číslo v proměnné `b` na velikost pole `a` a pak provede element-wise násobení.

Broadcasting se řídí dvěma jednoduchými pravidly. Porovnávájí se jednotlivé rozměry v `.shape` zprava doleva. Rozměry jsou kompatibilní, pokud:

1. jsou shodné
2. jeden z nich je 1.

Ukažme si to na několika příkladech.

In [ ]:
import numpy as np

a = np.array(range(5)).reshape((5, 1))
b = a.transpose()

print(a.shape, b.shape)

a * b

In [ ]:
# barevny obrazek velikosti IMG_SIZExIMG_SIZE. Kazdy barevny kanal chceme naskalovat zvlast

import numpy as np

IMG_SIZE = 4
im = np.array([1] * IMG_SIZE * IMG_SIZE * 3, dtype=int).reshape((IMG_SIZE, IMG_SIZE, 3))
scale = np.array([1, 2, 3])
scaled = im * scale
scaled

Pokud pole nemají _brodcastable_ rozměry, `numpy` vyvolá výjimku:

```{tip}
Broadcasting je zpočátku trochu matoucí. Asi nejsnazší je vyzkoušet si pár jednoduchých příkladů. Brzy se Vám to dostane pod kůži.
```

## Vektorizace

Velkým přínosem jsou vektorizované operace. Vektorizací se rozumí konverze algoritmu z opakovaných aplikací operace na prvky kolekce po jednom do podoby, ve které se operace provede pro všechny prvky najednou (nebo alespoň pro více najednou). Samotná realizace vektorizace se může velmi lišit v závislosti na kontextu. Cílem je především vyšší efektivita.

`numpy` právě se svými poli umožňuje provádět mnoho operací vektorizovaně, což je zpravidla významně rychlejší.

### Příklad

vezměme seznam 10001 čísel a spočítejme jejich odmocninu a přičteme číslo 1, 4 různými způsoby:

1. obyčejný for cyklus
2. numpy-vektorizovaná operace
3. hloupé nevyužití numpy pole
4. dodatečná vektorizace původně nevektorizované operace

Čas změříme pomocí magic metod balíčku timeit.

In [ ]:
import timeit
import numpy as np

def time_func(fun, number=1000, repeat=5):
    measurements = timeit.repeat(fun, number=number, repeat=repeat)
    mean = np.mean(measurements) * 1000 / number
    stdev = np.std(measurements) * 1000 / number
    print(f"function {fun.__name__} took ({mean:.3f}+-{stdev:.3f}) ms")

In [ ]:
from math import sqrt
import numpy as np

N = 10001

def basic_op():
    xs = [(0.0 + i *0.01) for i in range(N)]
    ys = [sqrt(x) + 1 for x in xs]

def np_op():
    x = np.linspace(0., 1., N)
    y = np.sqrt(x) + 1

def np_dumb_op():
    x = np.linspace(0., 1., N)
    y = np.zeros(x.shape, dtype=np.double)
    for i, xi in enumerate(x):
        y[i] = sqrt(xi) + 1


def on_el_op(x):
    return sqrt(x) + 1

vectorized_op = np.vectorize(on_el_op)

def vectorized_basic_op():
    x = np.linspace(0., 1., N)
    y = vectorized_op(x)


time_func(basic_op)
time_func(np_op)
time_func(np_dumb_op)
time_func(vectorized_basic_op)